<a href="https://colab.research.google.com/github/mpky/tb_colab/blob/master/ConvNet_sentdex_tutorial_20190417.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# download and unzip ngrok
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip

--2019-04-18 01:31:56--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 52.204.188.97, 52.45.248.161, 52.86.186.182, ...
Connecting to bin.equinox.io (bin.equinox.io)|52.204.188.97|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 14977695 (14M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip’

ngrok-stable-linux- 100%[===================>]  14.28M  8.49MB/s    in 1.7s    

2019-04-18 01:31:58 (8.49 MB/s) - ‘ngrok-stable-linux-amd64.zip’ saved [14977695/14977695]

Archive:  ngrok-stable-linux-amd64.zip
  inflating: ngrok                   


In [0]:
# run tensorboard
LOG_DIR = './log'
get_ipython().system_raw(
    'tensorboard --logdir {} --host 0.0.0.0 --port 6006 &'
    .format(LOG_DIR)
)

In [0]:
# run ngrok to tunnel tensorboard port 6006
get_ipython().system_raw('./ngrok http 6006 &')

In [4]:
# get the url for TB page
! curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

https://0c40c6d6.ngrok.io


In [5]:
# mount my drive
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.callbacks import TensorBoard
import pickle
import time

In [7]:
!pwd

/content


In [0]:
X = pickle.load(open("/content/gdrive/My Drive/Colab_Notebooks/Colab_Projects/sentdex_tutorial/X.pickle", "rb"))
y = pickle.load(open("/content/gdrive/My Drive/Colab_Notebooks/Colab_Projects/sentdex_tutorial/y.pickle", "rb"))

In [9]:
NAME = "Cats-vs-dog-cnn-64x2-50epochs-{}".format(int(time.time()))

tensorboard = TensorBoard(log_dir='./log/{}'.format(NAME))

# need to scale
X = X / 255

model = Sequential()
model.add(Conv2D(64, (3,3), input_shape=X.shape[1:]))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(64, (3,3)))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Flatten())
model.add(Dense(64))
model.add(Activation("relu"))

model.add(Dense(1))
model.add(Activation("sigmoid"))

model.compile(loss='binary_crossentropy',
             optimizer='adam',
             metrics=['accuracy'])

model.fit(X,y,batch_size=32,validation_split=0.3, epochs=20, callbacks=[tensorboard])

Instructions for updating:
Colocations handled automatically by placer.
Train on 17462 samples, validate on 7484 samples
Instructions for updating:
Use tf.cast instead.
Epoch 1/20
17462/17462 [==============================] - 8s 446us/sample - loss: 0.6436 - acc: 0.6144 - val_loss: 0.6185 - val_acc: 0.6646
Epoch 2/20
17462/17462 [==============================] - 4s 232us/sample - loss: 0.5317 - acc: 0.7390 - val_loss: 0.5242 - val_acc: 0.7487
Epoch 3/20
17462/17462 [==============================] - 4s 208us/sample - loss: 0.4939 - acc: 0.7628 - val_loss: 0.5020 - val_acc: 0.7592
Epoch 4/20
17462/17462 [==============================] - 4s 211us/sample - loss: 0.4586 - acc: 0.7835 - val_loss: 0.4932 - val_acc: 0.7647
Epoch 5/20
17462/17462 [==============================] - 4s 212us/sample - loss: 0.4310 - acc: 0.7990 - val_loss: 0.5195 - val_acc: 0.7441
Epoch 6/20
17462/17462 [==============================] - 4s 209us/sample - loss: 0.3971 - acc: 0.8167 - val_loss: 0.4803 - val_acc

In [10]:
dense_layers = [0,1,2]
layer_sizes = [32, 64, 128]
conv_layers = [1, 2, 3]

for dense_layer in dense_layers:
    for layer_size in layer_sizes:
        for conv_layer in conv_layers:
            NAME = "{}-conv-{}-nodes-{}-dense-{}".format(conv_layer, layer_size, dense_layer, int(time.time()))
            print(NAME)

            model = Sequential()
            model.add(Conv2D(layer_size, (3,3), input_shape=X.shape[1:]))
            model.add(Activation("relu"))
            model.add(MaxPooling2D(pool_size=(2,2)))

            for l in range(conv_layer-1):
              model.add(Conv2D(layer_size, (3,3)))
              model.add(Activation("relu"))
              model.add(MaxPooling2D(pool_size=(2,2)))

            model.add(Flatten())
            
            for l in range(dense_layer):
              model.add(Dense(layer_size))
              model.add(Activation("relu"))
            

            model.add(Dense(64))
            model.add(Activation("relu"))

            model.add(Dense(1))
            model.add(Activation("sigmoid"))

            tensorboard = TensorBoard(log_dir='./log/{}'.format(NAME))

            model.compile(loss='binary_crossentropy',
                         optimizer='adam',
                         metrics=['accuracy'])

            model.fit(X,y,batch_size=32,validation_split=0.3, 
                      epochs=10, 
                      callbacks=[tensorboard])

1-conv-32-nodes-0-dense-1555552342
Train on 17462 samples, validate on 7484 samples
Epoch 1/10
17462/17462 [==============================] - 3s 188us/sample - loss: 0.6935 - acc: 0.4994 - val_loss: 0.6934 - val_acc: 0.4881
Epoch 2/10
17462/17462 [==============================] - 3s 172us/sample - loss: 0.6932 - acc: 0.5051 - val_loss: 0.6925 - val_acc: 0.4881
Epoch 3/10
17462/17462 [==============================] - 3s 168us/sample - loss: 0.6590 - acc: 0.6184 - val_loss: 0.6432 - val_acc: 0.6438
Epoch 4/10
17462/17462 [==============================] - 3s 175us/sample - loss: 0.5787 - acc: 0.7083 - val_loss: 0.5730 - val_acc: 0.7086
Epoch 5/10
17462/17462 [==============================] - 3s 183us/sample - loss: 0.5180 - acc: 0.7466 - val_loss: 0.5727 - val_acc: 0.7177
Epoch 6/10
17462/17462 [==============================] - 3s 183us/sample - loss: 0.4758 - acc: 0.7737 - val_loss: 0.5571 - val_acc: 0.7273
Epoch 7/10
17462/17462 [==============================] - 3s 178us/sample - 

In [11]:
!ps

    PID TTY          TIME CMD
      1 ?        00:00:00 run.sh
      6 ?        00:00:04 node
     21 ?        00:00:14 node
     31 ?        00:00:45 jupyter-noteboo
    129 ?        00:00:00 tail
    142 ?        00:28:24 python3
    166 ?        00:00:25 tensorboard
    173 ?        00:00:08 ngrok
    213 ?        00:00:00 drive
    278 ?        00:00:01 drive
    327 ?        00:00:00 tail
    328 ?        00:00:00 grep
   1724 ?        00:00:00 ps


In [0]:
!kill 163 170

In [12]:
!ps

    PID TTY          TIME CMD
      1 ?        00:00:00 run.sh
      8 ?        00:00:01 node
     23 ?        00:00:03 node
     33 ?        00:00:07 jupyter-noteboo
    124 ?        00:00:00 tail
    139 ?        00:03:52 python3
    209 ?        00:00:00 drive
    275 ?        00:00:00 drive
    323 ?        00:00:00 tail
    324 ?        00:00:00 grep
    458 ?        00:00:00 ps
